<a href="https://colab.research.google.com/github/LondheShubham153/natural_language_processing/blob/main/topic_modelling_using_lda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Packages and Imports

Supressed Warnings

In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [ ]:
! pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 4.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=b66a4eb2b593582b01d7a786b827b4f79458666d8cc0d21cf1a2166ca87da06e
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
from nltk.corpus import stopwords
import string
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Loding Dataset

In [ ]:
newsgroups = fetch_20newsgroups(remove=('headers','footers','quotes'))
docs_raw = newsgroups.data

In [ ]:
print(f"The total number of Documents from the News Group Dataset is: {len(docs_raw)}")

The total number of Documents from the News Group Dataset is: 11314


## Case 1: CountVectorizer with Custom Function, TFIDF and BOW

Creating Custom Preprocessor

In [ ]:
class PreProcessor:

  def remove_punctuation(self, record):

    cleaned_str = [char for char in record if char not in string.punctuation]
    return ''.join(cleaned_str)
  
  def normalize_sentences(self, sentences):
    words = sentences.split(" ")
    return [word.lower() for word in words]

  def remove_stopwords(self,words):
    return [word for word in words if word not in stopwords.words("english")]

  def process(self, record):
      # Remove Punctuation
      sentences = self.remove_punctuation(record)
      
      # Normalize
      norm_words = self.normalize_sentences(sentences)
      
      # Remove Stopwords
      final_words = self.remove_stopwords(norm_words)
      
      return final_words

In [ ]:
processor = PreProcessor()

Using preprocessing and create BOW and TFIDF object

In [ ]:
word_vector = CountVectorizer(analyzer=processor.process)
final_word_vocab = word_vector.fit(docs_raw)

In [ ]:
bag_of_words = final_word_vocab.transform(docs_raw)

In [ ]:
tfIdf_obj = TfidfTransformer().fit(bag_of_words)
final_feature = tfIdf_obj.transform(bag_of_words)

Creating a TFIDF object for LDA

In [ ]:
tfidfVectorizerNew = TfidfVectorizer(**word_vector.get_params())
tfidfObjectNew = tfidfVectorizerNew.fit_transform(docs_raw)

In [ ]:
lda_tf_custom = LatentDirichletAllocation(n_components=20 , random_state=0)
lda_tf_custom.fit(tfidfObjectNew)

LatentDirichletAllocation(n_components=20, random_state=0)

Initializing Notebook

In [ ]:
pyLDAvis.enable_notebook()


In [ ]:
pyLDAvis.sklearn.prepare(lda_tf_custom,tfidfObjectNew,word_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.338611  0.002340       1        1  50.974763
3      0.031515  0.060537       2        1   5.224388
13     0.024236 -0.007688       3        1   3.737276
17     0.018807 -0.004480       4        1   2.762474
0      0.018449 -0.004817       5        1   2.731785
10     0.017560 -0.003066       6        1   2.506550
8      0.017348 -0.004108       7        1   2.485673
12     0.017333 -0.003759       8        1   2.471166
6      0.017102 -0.002480       9        1   2.417974
14     0.016721 -0.003783      10        1   2.400903
15     0.016399 -0.003484      11        1   2.352789
5      0.016013 -0.003116      12        1   2.343668
9      0.016564 -0.002817      13        1   2.330065
4      0.016216 -0.002882      14        1   2.277011
19     0.016446 -0.002912      15        1   2.275907
18     0.016331 -0.003201      16        1   2.246242
1      0.015959 -0.003039      17        1   2.222609
7      0.015782 -0.002794      18        1   2.180197
16     0.015601 -0.002673      19        1   2.156982
11     0.014229 -0.001776      20        1   1.901577, topic_info=            Term         Freq        Total Category  logprob  loglift
0                 1563.000000  1563.000000  Default  30.0000  30.0000
390           \n   136.000000   136.000000  Default  29.0000  29.0000
200685     would   110.000000   110.000000  Default  28.0000  28.0000
982         \n\n    32.000000    32.000000  Default  27.0000  27.0000
135170       one    95.000000    95.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
185911      tool     0.080696     1.257154  Topic20  -9.6896   1.2166
105505       irq     0.079069     1.256595  Topic20  -9.7099   1.1966
124632  moonroof     0.070251     0.426750  Topic20  -9.8282   2.1584
47647       bias     0.071250     0.651354  Topic20  -9.8141   1.7496
192839   vehicle     0.071065     1.505705  Topic20  -9.8167   0.9091

[989 rows x 6 columns], token_table=        Topic      Freq    Term
term                           
0           1  0.999814        
390         1  0.998027      \n
982         1  0.920733    \n\n
982        12  0.061382    \n\n
1172        1  0.924290  \n\n\n
...       ...       ...     ...
195776      1  0.990028     way
196491      1  1.001903    well
198918      2  0.691567   wings
200685      1  1.002493   would
201277      2  1.505692    wtte

[114 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 14, 18, 1, 11, 9, 13, 7, 15, 16, 6, 10, 5, 20, 19, 2, 8, 17, 12])

## Case 2: CountVectorizer without Custom Function

In [ ]:
tf_vectorizer = CountVectorizer(stop_words='english',
                               lowercase=True,
                               token_pattern= r'\b[a-zA-Z]{3,}',
                                max_df=0.5,
                                min_df=10,
                                strip_accents='unicode'
                               )

tfObject = tf_vectorizer.fit_transform(docs_raw)

Creating TFIDF object

In [ ]:
tfidfVectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
tfidfObject = tfidfVectorizer.fit_transform(docs_raw)

Applying LDA

20 Components

In [ ]:
lda_tf = LatentDirichletAllocation(n_components=20 , random_state=0)
lda_tf.fit(tfObject)

LatentDirichletAllocation(n_components=20, random_state=0)

10 Components

In [ ]:
lda_tfIDF = LatentDirichletAllocation(n_components=10 , random_state=0)
lda_tfIDF.fit(tfidfObject)

LatentDirichletAllocation(random_state=0)

PyLDAVis Visualizations

Visualizing 20 Topics

In [ ]:
pyLDAvis.sklearn.prepare(lda_tf,tfObject,tf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18    -0.101321 -0.113753       1        1  10.475539
14    -0.042851 -0.181076       2        1   9.432847
15    -0.128992 -0.116186       3        1   9.096305
8      0.169841  0.151550       4        1   6.835898
19     0.155137 -0.104808       5        1   6.496904
5      0.033919  0.024788       6        1   5.760279
0     -0.117427 -0.005486       7        1   5.523160
1     -0.087716  0.012618       8        1   5.512460
17     0.216159  0.001532       9        1   5.108008
10    -0.019289  0.102672      10        1   4.299145
13     0.102139 -0.044822      11        1   4.094620
11    -0.204570  0.075855      12        1   3.953350
3      0.095011 -0.102724      13        1   3.849773
12     0.125271 -0.121112      14        1   3.486034
7     -0.174781  0.022223      15        1   3.203124
4      0.158745 -0.081201      16        1   3.188767
6     -0.114493 -0.105063      17        1   3.155194
16    -0.089640  0.130976      18        1   2.896544
9      0.182321  0.266415      19        1   1.914609
2     -0.157464  0.187600      20        1   1.717440, topic_info=            Term         Freq        Total Category  logprob  loglift
5027         max  4675.000000  4675.000000  Default  30.0000  30.0000
3530         god  1971.000000  1971.000000  Default  29.0000  29.0000
7699       space  1275.000000  1275.000000  Default  28.0000  28.0000
3160        file  1729.000000  1729.000000  Default  27.0000  27.0000
2659         edu  2404.000000  2404.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
415        april    56.349466   537.981347  Topic20  -5.5227   1.8081
5633      office    50.691584   345.812725  Topic20  -5.6286   2.1442
5466         new    56.488855  2585.709164  Topic20  -5.5203   0.2406
8649  university    48.877971   630.113046  Topic20  -5.6650   1.5078
6210       power    48.897076  1076.645409  Topic20  -5.6646   0.9724

[1416 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
15        2  0.110355        abs
15       11  0.860770        abs
20        1  0.114664  absolutes
20       17  0.879091  absolutes
22        4  0.936778  abstracts
...     ...       ...        ...
9168      9  0.040207        zip
9168     13  0.013402        zip
9168     16  0.455683        zip
9168     18  0.013402        zip
9168     20  0.006701        zip

[6014 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 15, 16, 9, 20, 6, 1, 2, 18, 11, 14, 12, 4, 13, 8, 5, 7, 17, 10, 3])

Visualizing 10 Topics

In [ ]:
pyLDAvis.sklearn.prepare(lda_tfIDF,tfObject,tf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.105698  0.083409       1        1  32.084962
8      0.181443 -0.153864       2        1  27.497162
5      0.089444 -0.020077       3        1  26.279263
1      0.103413  0.152540       4        1  11.446114
3     -0.096051 -0.011617       5        1   0.826634
7     -0.080782 -0.034470       6        1   0.624272
4     -0.084446 -0.009971       7        1   0.385657
9     -0.084745  0.001109       8        1   0.338373
2     -0.066338 -0.002583       9        1   0.276985
0     -0.067637 -0.004476      10        1   0.240578, topic_info=         Term         Freq        Total Category  logprob  loglift
2659      edu  1240.000000  1240.000000  Default  30.0000  30.0000
8275   thanks  1607.000000  1607.000000  Default  29.0000  29.0000
4906     mail  1005.000000  1005.000000  Default  28.0000  28.0000
7166     sale   393.000000   393.000000  Default  27.0000  27.0000
3530      god  1428.000000  1428.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
229      alan     3.245659    48.465091  Topic10  -6.4115   3.3264
7507      sig     3.721742    83.320230  Topic10  -6.2746   2.9214
8263  testing     3.739717   105.187292  Topic10  -6.2698   2.6931
6592    raise     3.591231    90.100031  Topic10  -6.3103   2.8075
2659      edu     3.241950  1240.319393  Topic10  -6.4126   0.0829

[571 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
49        1  0.131483  accessories
49        2  0.065742  accessories
49        3  0.131483  accessories
49        4  0.065742  accessories
49        7  0.460191  accessories
...     ...       ...          ...
9141      1  0.352126        years
9141      2  0.119828        years
9141      3  0.354228        years
9141      4  0.171333        years
9141      6  0.001051        years

[2541 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 9, 6, 2, 4, 8, 5, 10, 3, 1])

In [ ]:
pyLDAvis.sklearn.prepare(lda_tf_custom,tfidfObjectNew,word_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.338611  0.002340       1        1  50.974763
3      0.031515  0.060537       2        1   5.224388
13     0.024236 -0.007688       3        1   3.737276
17     0.018807 -0.004480       4        1   2.762474
0      0.018449 -0.004817       5        1   2.731785
10     0.017560 -0.003066       6        1   2.506550
8      0.017348 -0.004108       7        1   2.485673
12     0.017333 -0.003759       8        1   2.471166
6      0.017102 -0.002480       9        1   2.417974
14     0.016721 -0.003783      10        1   2.400903
15     0.016399 -0.003484      11        1   2.352789
5      0.016013 -0.003116      12        1   2.343668
9      0.016564 -0.002817      13        1   2.330065
4      0.016216 -0.002882      14        1   2.277011
19     0.016446 -0.002912      15        1   2.275907
18     0.016331 -0.003201      16        1   2.246242
1      0.015959 -0.003039      17        1   2.222609
7      0.015782 -0.002794      18        1   2.180197
16     0.015601 -0.002673      19        1   2.156982
11     0.014229 -0.001776      20        1   1.901577, topic_info=            Term         Freq        Total Category  logprob  loglift
0                 1563.000000  1563.000000  Default  30.0000  30.0000
390           \n   136.000000   136.000000  Default  29.0000  29.0000
200685     would   110.000000   110.000000  Default  28.0000  28.0000
982         \n\n    32.000000    32.000000  Default  27.0000  27.0000
135170       one    95.000000    95.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
185911      tool     0.080696     1.257154  Topic20  -9.6896   1.2166
105505       irq     0.079069     1.256595  Topic20  -9.7099   1.1966
124632  moonroof     0.070251     0.426750  Topic20  -9.8282   2.1584
47647       bias     0.071250     0.651354  Topic20  -9.8141   1.7496
192839   vehicle     0.071065     1.505705  Topic20  -9.8167   0.9091

[989 rows x 6 columns], token_table=        Topic      Freq    Term
term                           
0           1  0.999814        
390         1  0.998027      \n
982         1  0.920733    \n\n
982        12  0.061382    \n\n
1172        1  0.924290  \n\n\n
...       ...       ...     ...
195776      1  0.990028     way
196491      1  1.001903    well
198918      2  0.691567   wings
200685      1  1.002493   would
201277      2  1.505692    wtte

[114 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 14, 18, 1, 11, 9, 13, 7, 15, 16, 6, 10, 5, 20, 19, 2, 8, 17, 12])

## Conclusion

- The model did well without Custom Function
- it took a long time to create the TFID Object
- When we observed the PyLDAVis for custom function with TFIDF and BOW, we saw overlapping Topics generated
- When we used CountVectorizer without any Custom PreProcessing functions, we saw better results